In [1]:
from pyspark.mllib.recommendation import MatrixFactorizationModel


In [2]:
from pyspark.mllib.recommendation import ALS

In [3]:
model = MatrixFactorizationModel.load(sc, "s3n://patricks3db/models")


In [4]:
userFeatures = model.userFeatures()
userFeatures.cache()
print userFeatures
productFeatures = model.productFeatures()
productFeatures.cache()
print productFeatures

PythonRDD[15] at RDD at PythonRDD.scala:43
PythonRDD[19] at RDD at PythonRDD.scala:43


In [5]:
count = 5000
user1 = 1573033844
user2 = 1587946258
top_ratings1 = model.recommendProducts(user1,count)
top_ratings2 = model.recommendProducts(user2,count)

In [6]:
#print top_ratings1
#print top_ratings2

In [7]:
power = 50

In [8]:
type(top_ratings1[0])

pyspark.mllib.recommendation.Rating

In [9]:
top_ratings1[0]._fields

('user', 'product', 'rating')

In [10]:
top_ratings1[0].product

1051338639

In [14]:
#intersect = [row for row in top_ratings1 if row.product in [ tr2.product for tr2 in top_ratings2]]
tr1 = dict({(row.product,row.rating) for row in top_ratings1})
tr2 = dict({(row.product,row.rating) for row in top_ratings2})
intersection = set(tr1.keys()).intersection(set(tr2.keys()))
print intersection

set([89925633, 910433154, 1605203075, 1275454340, 1144109061, 1519356678, 1875191176, 873271692, 1337356429, 833986702, 1428818831, 858813955, 987866901, 140543641, 1639145880, 1971356313, 1529229210, 1916215919, 350002587, 1478325402, 815507998, 1146633797, 814403760, 45448226, 2004820515, 497300132, 349180749, 242381937, 682508456, 1463109801, 1119293610, 1079354162, 1904593885, 2123066672, 445210592, 677422514, 1043260595, 581047220, 1776164141, 934184633, 546292562, 1192437693, 268285760, 718805344, 64952770, 1637323461, 157690184, 868733130, 1923551522, 862915662, 542343631, 1923502800, 1170909487, 1079455570, 1780921427, 1153151690, 1487027158, 1129707736, 1964415450, 374205135, 172507484, 942410205, 399177054, 1294413408, 859262630, 195713084, 1103871266, 1009647027, 879491214, 207000125, 834353213, 368660721, 1887100277, 222320246, 1230993870, 69105786])


In [15]:
print intersection

set([89925633, 910433154, 1605203075, 1275454340, 1144109061, 1519356678, 1875191176, 873271692, 1337356429, 833986702, 1428818831, 858813955, 987866901, 140543641, 1639145880, 1971356313, 1529229210, 1916215919, 350002587, 1478325402, 815507998, 1146633797, 814403760, 45448226, 2004820515, 497300132, 349180749, 242381937, 682508456, 1463109801, 1119293610, 1079354162, 1904593885, 2123066672, 445210592, 677422514, 1043260595, 581047220, 1776164141, 934184633, 546292562, 1192437693, 268285760, 718805344, 64952770, 1637323461, 157690184, 868733130, 1923551522, 862915662, 542343631, 1923502800, 1170909487, 1079455570, 1780921427, 1153151690, 1487027158, 1129707736, 1964415450, 374205135, 172507484, 942410205, 399177054, 1294413408, 859262630, 195713084, 1103871266, 1009647027, 879491214, 207000125, 834353213, 368660721, 1887100277, 222320246, 1230993870, 69105786])


In [16]:
print max(tr1.values())
print max(tr2.values())
intersect = set(tr1.keys()).intersection(set(tr2.keys()))
tr1 = {myKey: tr1[myKey] for myKey in intersect}
tr2 = {myKey: tr2[myKey] for myKey in intersect}


16.6601073159
198.56956599
13.5731431012
136.065443937


In [19]:
finalList = { myKey: (tr1[myKey]/max(tr1.values())*(100-power)/100,tr2[myKey]/max(tr2.values())*power/100) for myKey in intersect}

In [32]:
print len(finalList)
print finalList.items()

76
[(89925633, (0.24548054345617745, 0.27176116648076376)), (910433154, (0.2235181309581141, 0.2894705981272095)), (1605203075, (0.2226974889162284, 0.37322836413466653)), (1275454340, (0.21361823675978897, 0.39072396142423194)), (1144109061, (0.2646891621016699, 0.3065939445936651)), (1519356678, (0.2108686239760137, 0.2566536898153602)), (1009647027, (0.22151746440001857, 0.2633346379260522)), (1875191176, (0.3960276405990232, 0.29335425003051496)), (1923551522, (0.22032837854563772, 0.2603216288008187)), (873271692, (0.22783878740435817, 0.28958097642766145)), (1337356429, (0.22334697244046425, 0.4498140034827236)), (833986702, (0.23953431252771865, 0.2770504233151177)), (1428818831, (0.22046579416815063, 0.4104417354648346)), (207000125, (0.2786590824551629, 0.2939068757516565)), (858813955, (0.24734590651402036, 0.3252070197707472)), (987866901, (0.21455496577410713, 0.3461418539759354)), (140543641, (0.2637159149718825, 0.2670544306426871)), (1639145880, (0.2368594374201125, 0.37

In [37]:
sortedfinallist = sorted(finalList.items(), key = lambda item: min(item[1][0],item[1][1]), reverse=True)
print sortedfinallist

[(2004820515, (0.3490057359201554, 0.3441851576368279)), (934184633, (0.5, 0.33824757514073395)), (1463109801, (0.3444619544507442, 0.33052255837816075)), (677422514, (0.3078735407009107, 0.3659125574737407)), (868733130, (0.30680926821460797, 0.3798653507545399)), (242381937, (0.3060683609923914, 0.3762932162912086)), (1478325402, (0.2960133993881086, 0.3436488318722953)), (1079455570, (0.29576398111263774, 0.3553122762517896)), (1875191176, (0.3960276405990232, 0.29335425003051496)), (581047220, (0.3531684369968902, 0.2875044522384629)), (368660721, (0.348300851150509, 0.28664049375397105)), (1529229210, (0.28284960277472737, 0.2792766284446825)), (207000125, (0.2786590824551629, 0.2939068757516565)), (862915662, (0.4305537008001876, 0.2689506566059703)), (1230993870, (0.26515516588668014, 0.3619285582383799)), (1144109061, (0.2646891621016699, 0.3065939445936651)), (140543641, (0.2637159149718825, 0.2670544306426871)), (1119293610, (0.2636564495616598, 0.2643074878882508)), (8155079

In [46]:
idlist = [str(x[0]) for x in sortedfinallist[0:12]]
print idlist
myString = ",".join(idlist)
print myString

['2004820515', '934184633', '1463109801', '677422514', '868733130', '242381937', '1478325402', '1079455570', '1875191176', '581047220', '368660721', '1529229210']
2004820515,934184633,1463109801,677422514,868733130,242381937,1478325402,1079455570,1875191176,581047220,368660721,1529229210


In [47]:
from cassandra.cluster import Cluster
#importing Cassandra modules from the driver we just installed
from cqlengine import connection
from cassandra.cluster import Cluster
CASSANDRA_KEYSPACE = "playground"
# setting up connections to cassandra
cluster = Cluster(['ec2-52-88-117-89.us-west-2.compute.amazonaws.com'])
session = cluster.connect('playground')

In [51]:
stmt = "select * from movie_catalog2 where  pid in (" + myString + ")"
response = session.execute(stmt)
print response
#shuffle(response)
output = ""

[Row(pid=242381937, asin=u'B00CUSEU42', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/5105bk2VTjL._SY300_.jpg', price=25.95, title=None), Row(pid=368660721, asin=u'B000286RRA', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/51SC6AZ4BNL._SY300_.jpg', price=5.98, title=u'Samson'), Row(pid=581047220, asin=u'B001B187GQ', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/51tv%2BCq57-L._SY300_.jpg', price=19.95, title=None), Row(pid=677422514, asin=u'B000NBBF7I', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/317gZ4mynEL.jpg', price=5.0, title=None), Row(pid=868733130, asin=u'B00799I57A', brand=None, imUrl=u'http://ecx.images-amazon.com/images/I/51ivrRwl2fL._SY300_.jpg', price=202.47, title=None), Row(pid=934184633, asin=u'0767011015', brand=u'', imUrl=u'http://ecx.images-amazon.com/images/I/41770H1EJ5L._SY300_.jpg', price=2.73, title=u'Avengers: Bird Who Knew Too Much &amp; Winged Avenger [VHS]'), Row(pid=1079455570, asin=u'B000MAM7FE', brand=None, 